In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
import sklearn
sklearn.__version__

'0.21.2'

In [2]:
path = '~/data/tinkoff/'

cust = pd.read_csv(path + 'customer_test.csv')
transactions = pd.read_csv(path + 'transactions.csv')
react_train = pd.read_csv(path + 'stories_reaction_train.csv')
react_test = pd.read_csv(path + 'stories_reaction_test.csv')
descr = pd.read_csv(path + 'stories_description.csv')
sample = pd.read_csv(path + 'sample_submit.csv')

In [3]:
import sys
sys.path.insert(1, '../scripts')

In [4]:
from preprocessing import preprocess_customer, preprocess_reactions

In [6]:
new_cust = preprocess_customer(cust, transactions, react_train, encodings=['mean', 'frequency'])

In [12]:
train, test = preprocess_reactions(react_train, react_test, encodings = ['mean', 'frequency', 'std'])

like, view, skip, dislike =  [1 3 2 0]


0 - dislike
1 - like
2 - skip
3 - view

In [11]:
from xgboost import XGBClassifier

In [12]:
data = train.join(new_cust.set_index('customer_id'), on = 'customer_id')
data_test = test.join(new_cust.set_index('customer_id'), on = 'customer_id')

In [27]:
y_train = data['event']
data.drop('event', axis=1, inplace=True)

In [28]:
xgb = XGBClassifier()

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data, y_train)

In [32]:
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [18]:
from sklearn.metrics import accuracy_score

In [33]:
y_val_pred = xgb.predict(X_val)
accuracy_score(y_val, y_val_pred)

0.7055695517643678

In [36]:
y_test = xgb.predict_proba(data_test)

In [40]:
def choose(y_test):
    p_dislike = y_test[:, 0]
    p_like = y_test[:, 1]
    p_skip = y_test[:, 2]
    p_view = y_test[:, 3]
    
    exp = -10 * p_dislike - 0.1 * p_skip + 0.1 * p_view + 0.5 * p_like
    res = pd.Series(exp).apply(lambda x: 1 if x > 0 else -1)
    return res

In [41]:
res = choose(y_test)

In [43]:
sample['score'] = res

In [44]:
sample.to_csv('../submissions/defaultxgboost.csv', index = False)